In [1]:
import sys
sys.path.append('..')
from src.utilities import *
from src.models.train_model import train_model
from src.models.param_opt import bayes_parameter_opt_lgb
from sklearn.model_selection import KFold

## 3. Modelling

### 3.1 Load master table

In [2]:
master = pd.read_csv(os.path.join(processed_path, 'master.csv'))

### 3.2 Train / val / test split

In [3]:
x_train = master[(master.sales.isna()==False)].drop(columns = ['region', 'brand']).copy()
x_train = x_train[x_train.month >= '2020-06']
x_train['train'] =  (x_train.month <= '2021-06').astype(float)
x_train.drop(columns = 'month', inplace = True)

### 3.3 Bayesian parameter search

In [4]:
# best_params = bayes_parameter_opt_lgb(X = x_train.drop(columns = ['sales', 'train']), y = x_train.sales,
#                                       init_round=30, opt_round=30, n_folds=5, random_seed=6, n_estimators=10000, 
#                                       learning_rate=0.01, save_path = '', metric = 'mae').max['params']
# best_params

### 3.4 Model training

In [5]:
##### Define best parameters found earlier
best_params = {'metric': 'mae',
               'bagging_fraction':  0.8007,
               'feature_fraction': 0.5893,
               'lambda_l1': 1.727,
               'lambda_l2': 1.436,
               'max_depth': 17,
               'min_child_weight': 13.77,
               'min_split_gain': 0.0331  ,
               'num_leaves': 44}

kf = KFold(n_splits=5, shuffle=True, random_state=42)
lgb_quantiles = {}
fold = 0

for train_index, test_index in kf.split(x_train):
    ##### Train a quantile regression lgb at different alpha levels #####
    quantile_alphas = [0.1, 0.5, 0.9]
    models = {}
    for alpha in quantile_alphas:
        current_model = train_model(x_train.iloc[train_index], target_name = 'sales', 
                                    model_type = 'lgb-quantile', 
                                    quantile_alpha = alpha,
                                    params = best_params, 
                                    metric = 'rmse', 
                                    split = 'in_sample',
                                    save_path = os.path.join(models_path, 
                                                             'model2_quantile_'+ str(alpha)+'.pkl'))
        models[alpha] = current_model
    lgb_quantiles[fold] = models
    fold += 1

####################     training with  3261      ####################
####################     validating with  363      ####################
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.042948 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 71975
[LightGBM] [Info] Number of data points in the train set: 3261, number of used features: 865
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l1: 1797.8	valid_1's l1: 1852.02
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's l1: 1780.21	valid_1's l1: 1834.59
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	training's l1: 1761.5	valid_1's l1: 1817.15
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	training's l1: 1749.1	valid_1's l1: 1805.1
[LightGBM] 

[149]	training's l1: 1191.38	valid_1's l1: 1296.26
[150]	training's l1: 1191.14	valid_1's l1: 1296.04
[151]	training's l1: 1190.95	valid_1's l1: 1295.94
[152]	training's l1: 1190.78	valid_1's l1: 1296
[153]	training's l1: 1190.68	valid_1's l1: 1295.95
[154]	training's l1: 1190.08	valid_1's l1: 1295.53
[155]	training's l1: 1190.02	valid_1's l1: 1295.47
[156]	training's l1: 1189.94	valid_1's l1: 1295.45
[157]	training's l1: 1189.49	valid_1's l1: 1295.2
[158]	training's l1: 1188.21	valid_1's l1: 1293.67
[159]	training's l1: 1188.03	valid_1's l1: 1293.55
[160]	training's l1: 1187.61	valid_1's l1: 1293.19
[161]	training's l1: 1186.66	valid_1's l1: 1292.76
[162]	training's l1: 1186.22	valid_1's l1: 1292.35
[163]	training's l1: 1186.14	valid_1's l1: 1292.35
[164]	training's l1: 1185.47	valid_1's l1: 1291.51
[165]	training's l1: 1185.03	valid_1's l1: 1291.33
[166]	training's l1: 1184.91	valid_1's l1: 1291.27
[167]	training's l1: 1184.75	valid_1's l1: 1291.17
[168]	training's l1: 1184.69	valid_

[305]	training's l1: 1128.14	valid_1's l1: 1248.05
[306]	training's l1: 1128.09	valid_1's l1: 1248.08
[307]	training's l1: 1128.02	valid_1's l1: 1248
[308]	training's l1: 1127.68	valid_1's l1: 1247.69
[309]	training's l1: 1127.62	valid_1's l1: 1247.66
[310]	training's l1: 1127.44	valid_1's l1: 1247.5
[311]	training's l1: 1127.36	valid_1's l1: 1247.39
[312]	training's l1: 1127.31	valid_1's l1: 1247.34
[313]	training's l1: 1127.24	valid_1's l1: 1247.29
[314]	training's l1: 1127.21	valid_1's l1: 1247.29
[315]	training's l1: 1127.12	valid_1's l1: 1247.28
[316]	training's l1: 1127.09	valid_1's l1: 1247.29
[317]	training's l1: 1126.81	valid_1's l1: 1247.1
[318]	training's l1: 1126.61	valid_1's l1: 1247.04
[319]	training's l1: 1126.48	valid_1's l1: 1246.9
[320]	training's l1: 1126.41	valid_1's l1: 1246.83
[321]	training's l1: 1126.35	valid_1's l1: 1246.8
[322]	training's l1: 1126.28	valid_1's l1: 1246.75
[323]	training's l1: 1126.06	valid_1's l1: 1246.63
[324]	training's l1: 1126.03	valid_1's

[463]	training's l1: 1102.44	valid_1's l1: 1227.8
[464]	training's l1: 1102.41	valid_1's l1: 1227.78
[465]	training's l1: 1102.38	valid_1's l1: 1227.76
[466]	training's l1: 1102.33	valid_1's l1: 1227.71
[467]	training's l1: 1102.29	valid_1's l1: 1227.68
[468]	training's l1: 1102.24	valid_1's l1: 1227.66
[469]	training's l1: 1102.11	valid_1's l1: 1227.52
[470]	training's l1: 1102.1	valid_1's l1: 1227.51
[471]	training's l1: 1102.02	valid_1's l1: 1227.44
[472]	training's l1: 1101.97	valid_1's l1: 1227.45
[473]	training's l1: 1101.64	valid_1's l1: 1227.29
[474]	training's l1: 1101.59	valid_1's l1: 1227.25
[475]	training's l1: 1101.42	valid_1's l1: 1227.2
[476]	training's l1: 1101.37	valid_1's l1: 1227.17
[477]	training's l1: 1101.35	valid_1's l1: 1227.15
[478]	training's l1: 1101.08	valid_1's l1: 1227.14
[479]	training's l1: 1101.07	valid_1's l1: 1227.14
[480]	training's l1: 1100.77	valid_1's l1: 1227.02
[481]	training's l1: 1100.75	valid_1's l1: 1227.02
[482]	training's l1: 1100.45	valid

[604]	training's l1: 1087.52	valid_1's l1: 1216.94
[605]	training's l1: 1087.5	valid_1's l1: 1216.93
[606]	training's l1: 1087.47	valid_1's l1: 1216.93
[607]	training's l1: 1087.44	valid_1's l1: 1216.9
[608]	training's l1: 1087.39	valid_1's l1: 1216.87
[609]	training's l1: 1087.36	valid_1's l1: 1216.85
[610]	training's l1: 1087.29	valid_1's l1: 1216.76
[611]	training's l1: 1087.26	valid_1's l1: 1216.72
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[612]	training's l1: 1087.18	valid_1's l1: 1216.72
[613]	training's l1: 1087.17	valid_1's l1: 1216.71
[614]	training's l1: 1087.06	valid_1's l1: 1216.72
[615]	training's l1: 1086.99	valid_1's l1: 1216.66
[616]	training's l1: 1086.26	valid_1's l1: 1216.12
[617]	training's l1: 1086.24	valid_1's l1: 1216.12
[618]	training's l1: 1086.18	valid_1's l1: 1216.1
[619]	training's l1: 1086.17	valid_1's l1: 1216.08
[620]	training's l1: 1086.13	valid_1's l1: 1216.06
[621]	training's l1: 1086.12	valid_1's l1: 1216.05
[622]	trai

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1256]	training's l1: 735.457	valid_1's l1: 1086.57
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1257]	training's l1: 735.456	valid_1's l1: 1086.57
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1258]	training's l1: 735.451	valid_1's l1: 1086.57
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1259]	training's l1: 735.431	valid_1's l1: 1086.55
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1260]	training's l1: 735.433	valid_1's l1: 1086.55
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1261]	training's l1: 735.422	valid_1's l1: 1086.55
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1262]	training's l1: 735.398	valid_1's l1: 1086.53
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1263]	training's l1: 735.382	valid_

[1354]	training's l1: 733.047	valid_1's l1: 1084.94
[1355]	training's l1: 733.009	valid_1's l1: 1084.91
[1356]	training's l1: 732.99	valid_1's l1: 1084.9
[1357]	training's l1: 732.98	valid_1's l1: 1084.9
[1358]	training's l1: 732.973	valid_1's l1: 1084.9
[1359]	training's l1: 732.968	valid_1's l1: 1084.89
[1360]	training's l1: 732.959	valid_1's l1: 1084.89
[1361]	training's l1: 732.861	valid_1's l1: 1084.72
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1362]	training's l1: 732.841	valid_1's l1: 1084.7
[1363]	training's l1: 732.837	valid_1's l1: 1084.7
[1364]	training's l1: 732.832	valid_1's l1: 1084.7
[1365]	training's l1: 732.824	valid_1's l1: 1084.69
[1366]	training's l1: 732.815	valid_1's l1: 1084.69
[1367]	training's l1: 732.809	valid_1's l1: 1084.68
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1368]	training's l1: 732.799	valid_1's l1: 1084.67
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[13

[1472]	training's l1: 729.023	valid_1's l1: 1081.5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1473]	training's l1: 729.002	valid_1's l1: 1081.5
[1474]	training's l1: 728.993	valid_1's l1: 1081.49
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1475]	training's l1: 728.991	valid_1's l1: 1081.49
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1476]	training's l1: 728.983	valid_1's l1: 1081.49
[1477]	training's l1: 728.976	valid_1's l1: 1081.49
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1478]	training's l1: 728.96	valid_1's l1: 1081.47
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1479]	training's l1: 728.951	valid_1's l1: 1081.47
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1480]	training's l1: 728.932	valid_1's l1: 1081.45
[1481]	training's l1: 728.821	valid_1's l1: 1081.29
[1482]	training's l1: 728.632	val

[1580]	training's l1: 724.95	valid_1's l1: 1078.32
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1581]	training's l1: 724.954	valid_1's l1: 1078.33
[1582]	training's l1: 724.784	valid_1's l1: 1078.24
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1583]	training's l1: 724.65	valid_1's l1: 1078.24
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1584]	training's l1: 724.64	valid_1's l1: 1078.23
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1585]	training's l1: 724.64	valid_1's l1: 1078.23
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1586]	training's l1: 724.616	valid_1's l1: 1078.21
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1587]	training's l1: 724.604	valid_1's l1: 1078.21
[1588]	training's l1: 724.583	valid_1's l1: 1078.2
[1589]	training's l1: 724.571	valid_1's l1: 1078.19
[1590]	training's l1: 724.561	valid

[1719]	training's l1: 721.369	valid_1's l1: 1075.71
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1720]	training's l1: 721.346	valid_1's l1: 1075.69
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1721]	training's l1: 721.339	valid_1's l1: 1075.69
[1722]	training's l1: 721.305	valid_1's l1: 1075.65
[1723]	training's l1: 721.308	valid_1's l1: 1075.66
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1724]	training's l1: 721.302	valid_1's l1: 1075.65
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1725]	training's l1: 721.301	valid_1's l1: 1075.65
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1726]	training's l1: 721.303	valid_1's l1: 1075.65
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1727]	training's l1: 721.281	valid_1's l1: 1075.63
[1728]	training's l1: 721.262	valid_1's l1: 1075.63
[1729]	training's l1: 721.167	

### 3.5 Prediction storage

In [6]:
submission = master[(master.sales.isna()) & (master.month >= '2020-07')].copy()

In [11]:
lower = (lgb_quantiles[0][0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[1][0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[2][0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[3][0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[4][0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))) / 5 

upper = (lgb_quantiles[0][0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[1][0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[2][0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[3][0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[4][0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))) / 5 

sales = (lgb_quantiles[0][0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[1][0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[2][0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[3][0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales'])) +
         lgb_quantiles[4][0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))) / 5 

submission['sales'] = sales
submission['lower'] = lower
submission['upper'] = upper

# submission['sales'] = lgb_quantiles[0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
# submission['lower'] = lgb_quantiles[0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
# submission['upper'] = lgb_quantiles[0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales', 'lower']))

submission = submission[['month', 'region', 'brand', 'sales', 'lower', 'upper']]

submission.loc[submission.sales < 0, 'sales'] = 0
submission.loc[submission.lower < 0, 'lower'] = 0
submission.loc[submission.upper < 0, 'upper'] = 0

submission.loc[submission.lower > submission.sales, 'lower'] = submission.sales
submission.loc[submission.upper < submission.sales, 'upper'] = submission.sales

submission.head()

,month,region,brand,sales,lower,upper
2714,2020-07,region_151,brand_1,15.572264,0.000000,113.776544
2715,2020-07,region_151,brand_2,15.572264,0.000000,113.776544
2716,2020-07,region_152,brand_1,75.489429,0.057055,422.066896
2717,2020-07,region_152,brand_2,75.489429,0.057055,422.066896
2718,2020-07,region_153,brand_1,35.387825,0.000000,831.884228


In [13]:
#submission.to_csv(os.path.join(results_path, 'submission14_team46.csv'), index = False)

In [15]:
(submission['upper'] - submission['lower']).mean()

1849.1844372387657

In [9]:
preds_train = pd.DataFrame({'month': master[(master.sales.isna()==False) & (master.month >= '2020-07')].month,
                            'region': master[(master.sales.isna()==False) & (master.month >= '2020-07')].region,
                            'brand': master[(master.sales.isna()==False) & (master.month >= '2020-07')].brand,
                            'predictions': lgb_quantiles[0.5].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'lower': lgb_quantiles[0.1].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'upper': lgb_quantiles[0.9].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'sales': master[(master.sales.isna()==False) & (master.month >= '2020-07')].sales
                           })

preds_train.loc[preds_train.predictions < 0, 'predictions'] = 0
preds_train.loc[preds_train.lower < 0, 'lower'] = 0
preds_train.loc[preds_train.upper < 0, 'upper'] = 0

width = abs(preds_train['upper'] - preds_train['lower'])
below = (preds_train['lower'] - preds_train['sales'])*(preds_train['lower'] > preds_train['sales'])
above = (preds_train['sales'] - preds_train['upper'])*(preds_train['upper'] < preds_train['sales'])
outside = (2/0.2)*(below + above)

In [10]:
preds_train

,month,region,brand,predictions,lower,upper,sales
2412,2020-07,region_0,brand_1,31.791915,0.000000,164.680759,0.00
2413,2020-07,region_0,brand_2,31.791915,0.000000,164.680759,0.00
2414,2020-07,region_1,brand_1,135.803402,0.000000,499.677776,0.00
2415,2020-07,region_1,brand_2,135.803402,0.000000,499.677776,143.74
2416,2020-07,region_2,brand_1,27.882901,0.000000,77.049651,0.00
...,...,...,...,...,...,...,...
7935,2021-08,region_148,brand_2,5690.742390,1573.813731,5635.445018,4777.64
7936,2021-08,region_149,brand_1,2598.026795,1355.877009,3956.231328,2762.08
7937,2021-08,region_149,brand_2,721.880225,677.627469,1140.014869,998.29
7938,2021-08,region_150,brand_1,5749.099298,3228.418285,6621.892190,5634.64


In [11]:
preds_train['outside'] = (preds_train.sales < preds_train.lower)  | (preds_train.sales > preds_train.upper)
preds_train.outside.mean()

0.1620151371807001

In [11]:
width.mean()
# 0.25 988.73
# 0.2  1090.40
# 0.1 1827.48

1771.7814207767412

In [12]:
outside.mean()
# 0.25 1276.75
# 0.2  632.81
# 0.1 209.00

237.28347669251508

In [13]:
(width + outside).mean()
# 0.25 2265.492
# 0.2  1723.220
# 0.1 2036.48

2009.0648974692563